In [1]:
#import importlib
#importlib.reload(_prepare)

In [2]:
import warnings
warnings.filterwarnings(action='once')


Original idea from: https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/


In this post, we discuss techniques to visualize the output and results from topic model (LDA) based on the gensim package. I will be using a portion of the 20 Newsgroups dataset since the focus is more on approaches to visualizing the results.

Let’s begin by importing the packages and the 20 News Groups dataset.

In [3]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['linkremoved',' <link removed>','usernameremoved','<usernameremoved>','<linkremoved>','usernameremoved_usernameremoved','linkremoved_linkremoved'])


%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

c:\users\gonza\tesisenv\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\users\gonza\tesisenv\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
c:\users\gonza\tesisenv\lib\site-packages\thinc\neural\_custom_kernels.py:36: ResourceWarning: unclosed file <_io.TextIOWrapper name='c:\\users\\gonza\\tesisenv\\lib\\site-packages\\thinc\\neural\\_custom_kernels.cu' mode='r' encoding='utf8'>
  SRC = (PWD / "_custom_kernels.cu").open("r", encoding="utf8").read()
c:\users\gonza\tesisenv\lib\site-packages\thinc\neural\_custom_kernels.py:39: ResourceWarn

In [4]:
#Instructions about how to install mallet are available here: http://mallet.cs.umass.edu/download.php

'''
Windows installation: After unzipping MALLET, set the environment variable %MALLET_HOME% to point to the MALLET directory.
In all command line examples, substitute bin\mallet for bin/mallet.
'''
import os
from gensim.models.wrappers import LdaMallet


path_to_mallet_binary = "C:\\mallet-2.0.8\\bin\\mallet"
os.environ.update({'MALLET_HOME':r'C:\mallet-2.0.8'}) #OJO!, por alguna razon mallet solo puede estar disponible en esa carpeta


## Import Cambridge Analytica datasets


In [5]:
#this libraries are for this dataset
import pandas as pd


In [6]:
file_name_1 = 'english_europe_tweets_20190411.csv'
file_name_2 = 'english_northamerica_tweets_20190411.csv'


In [7]:
df_1  = pd.read_csv('data/cambridge_analytica/regional_datasets/'+file_name_1)
df_2  = pd.read_csv('data/cambridge_analytica/regional_datasets/'+file_name_2)

In [8]:
#df_1 = df_1.sample(1000)  #remove this line
#df_2 = df_2.sample(1000)  #remove this line

In [9]:
df_1.head()

,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,2,2018-04-01,id-980480940567924736,Sun Apr 01 16:24:07 +0000 2018,CamHatch,"Rotterdam, The Netherlands",NaN,Meet our newest product: the transparent CamHa...,Meet our newest product: the transparent CamHa...,NaN,NaN,Meet our newest product: the transparent CamHa...,Netherlands
1,17,2018-04-01,id-980481203110281216,Sun Apr 01 16:25:09 +0000 2018,iky86,stoke on trent,NaN,@SupportOurLefty This week has been the most #...,@SupportOurLefty This week has been the most #...,NaN,NaN,<usernameremoved> This week has been the most ...,United Kingdom
2,43,2018-04-01,id-980480454846566401,Sun Apr 01 16:22:11 +0000 2018,MichaelJF80,"Liverpool, England",NaN,Pulitzer prize for this lady. Investigative jo...,NaN,NaN,NaN,Pulitzer prize for this lady. Investigative jo...,United Kingdom
3,45,2018-04-01,id-980480424429400071,Sun Apr 01 16:22:04 +0000 2018,ruechids1,"Enfield, London",NaN,"Theres no more secrecy, confident and privacy ...","Theres no more secrecy, confident and privacy ...",NaN,NaN,"Theres no more secrecy, confident and privacy ...",United Kingdom
4,51,2018-04-01,id-980480325083164673,Sun Apr 01 16:21:40 +0000 2018,Hugh_Macfarlane,"Glenrothes, Fife",NaN,AggregateIQ: the obscure Canadian tech firm an...,NaN,NaN,NaN,AggregateIQ: the obscure Canadian tech firm an...,United Kingdom


In [10]:
df_2.head()

,Unnamed: 0,created_at2,id,created_at,user.screen_name,user_location,coordinates,text,full_text,retweet_created_at,retweet_full_text,texto_completo,ubicacion_encontrada
0,1,2018-04-01,id-980480936205783047,Sun Apr 01 16:24:06 +0000 2018,stanscott53,"Coldspring, TX",NaN,@SenTedCruz Hold Facebook accountable for abus...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States
1,6,2018-04-01,id-980481012550524928,Sun Apr 01 16:24:24 +0000 2018,RShaffer1,"Boston, MA",NaN,Both sides should just drop it. Trump won...el...,NaN,NaN,NaN,Both sides should just drop it. Trump won...el...,United States
2,9,2018-04-01,id-980480980191404032,Sun Apr 01 16:24:16 +0000 2018,pcampisi14,"Toronto, Ontario",NaN,$AAPL lobbing threes while they can. Wasnt it ...,$AAPL lobbing threes while they can. Wasnt it ...,NaN,NaN,$AAPL lobbing threes while they can. Wasnt it ...,Canada
3,16,2018-04-01,id-980481205492682753,Sun Apr 01 16:25:10 +0000 2018,CoinSignalBot,"Seattle, WA",NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,NaN,NaN,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...,United States
4,20,2018-04-01,id-980481164334063616,Sun Apr 01 16:25:00 +0000 2018,kimbee802009,"South Dakota, USA",NaN,@SenJohnThune Hold Facebook accountable for ab...,NaN,NaN,NaN,<usernameremoved> Hold Facebook accountable fo...,United States


## Tokenize Sentences and Clean

Removing the emails, new line characters, single quotes and finally split the sentence into a list of words using gensim’s simple_preprocess(). Setting the deacc=True option removes punctuations.

In [11]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  



In [12]:
# Convert to list
data_1 = df_1.texto_completo.values.tolist()
data_words_1 = list(sent_to_words(data_1))
print(data_words_1[:1])

data_2 = df_2.texto_completo.values.tolist()
data_words_2 = list(sent_to_words(data_2))
print(data_words_2[:2])


[['meet', 'our', 'newest', 'product', 'the', 'transparent', 'camhatch', 'webcam', 'cover', 'weve', 'combined', 'everything', 'you', 'loved', 'about', 'the', 'matte', 'black', 'version', 'with', 'the', 'transparency', 'of', 'mark', 'zuckerberg', 'protect', 'your', 'privacy', 'without', 'having', 'to', 'change', 'the', 'appearance', 'of', 'your', 'laptop', 'tablet', 'or', 'smartphone', 'link', 'removed']]
[['usernameremoved', 'hold', 'facebook', 'accountable', 'for', 'abusing', 'our', 'data', 'and', 'censoring', 'our', 'voice', 'vote', 'no', 'on', 'the', 'cra', 'link', 'removed'], ['both', 'sides', 'should', 'just', 'drop', 'it', 'trump', 'won', 'elections', 'over', 'move', 'on', 'link', 'removed']]


## 4. Build the Bigram, Trigram Models and Lemmatize


Let’s form the bigram and trigrams using the Phrases model. This is passed to Phraser() for efficiency in speed of execution.

Next, lemmatize each word to its root form, keeping only nouns, adjectives, verbs and adverbs.

We keep only these POS tags because they are the ones contributing the most to the meaning of the sentences. Here, I use spacy for lemmatization.

In [13]:
#In case you haven't installed yet
#!python -m spacy download en_core_web_sm

In [14]:
# Build the bigram and trigram models
bigram_1 = gensim.models.Phrases(data_words_1, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_1 = gensim.models.Phrases(bigram_1[data_words_1], threshold=100)  
bigram_mod_1 = gensim.models.phrases.Phraser(bigram_1)
trigram_mod_1 = gensim.models.phrases.Phraser(trigram_1)

bigram_2 = gensim.models.Phrases(data_words_2, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram_2 = gensim.models.Phrases(bigram_2[data_words_2], threshold=100)  
bigram_mod_2 = gensim.models.phrases.Phraser(bigram_2)
trigram_mod_2 = gensim.models.phrases.Phraser(trigram_2)

In [15]:

# !python3 -m spacy download en  # run in terminal once
"""Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
def process_words(texts, bigram_mod, trigram_mod, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):    
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []    
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out
    



In [16]:
data_ready_1 = process_words(data_words_1, bigram_mod_1, trigram_mod_1)  # processed Text Data!
data_ready_2 = process_words(data_words_2, bigram_mod_2, trigram_mod_2)  # processed Text Data!

# Build the topic model

To build the LDA topic model using LdaModel(), you need the corpus and the dictionary. Let’s create them first and then build the model. The trained topics (keywords and weights) are printed below as well.



## topic modeling europe dataset

In [17]:
#with genim and mallet
number_topics_1 = 11

In [18]:
# Create Dictionary
id2word_1 = corpora.Dictionary(data_ready_1)

# Create Corpus: Term Document Frequency
corpus_1 = [id2word_1.doc2bow(text) for text in data_ready_1]




In [19]:
lda_model_1 = LdaMallet(path_to_mallet_binary, corpus=corpus_1, num_topics=number_topics_1, id2word=id2word_1)

c:\users\gonza\tesisenv\lib\site-packages\gensim\models\wrappers\ldamallet.py:285: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\Users\\gonza\\AppData\\Local\\Temp\\a4df11_state.mallet.gz'>
  self.word_topics = self.load_word_topics()


In [20]:
#convert lda mallet model to lda gensim model
lda_model_1 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model_1)

In [21]:
pprint(lda_model_1.print_topics())

[(0,
  '0.318*"remove" + 0.288*"link" + 0.033*"late" + 0.023*"business" + '
  '0.020*"marketing" + 0.011*"learn" + 0.009*"daily" + 0.009*"advertising" + '
  '0.008*"customer" + 0.007*"tool"'),
 (1,
  '0.143*"remove" + 0.127*"link" + 0.093*"facebook" + 0.038*"time" + '
  '0.024*"news" + 0.024*"ad" + 0.016*"platform" + 0.015*"story" + 0.013*"real" '
  '+ 0.012*"show"'),
 (2,
  '0.094*"privacy" + 0.049*"facebook" + 0.024*"change" + 0.023*"scandal" + '
  '0.020*"law" + 0.019*"data" + 0.014*"issue" + 0.012*"problem" + '
  '0.011*"policy" + 0.011*"user"'),
 (3,
  '0.190*"datum" + 0.051*"user" + 0.030*"share" + 0.028*"company" + '
  '0.027*"give" + 0.022*"facebook" + 0.020*"personal" + 0.018*"information" + '
  '0.017*"data" + 0.016*"app"'),
 (4,
  '0.371*"remove" + 0.356*"link" + 0.029*"read" + 0.017*"interesting" + '
  '0.012*"article" + 0.011*"important" + 0.009*"research" + 0.006*"thread" + '
  '0.005*"piece" + 0.005*"testify"'),
 (5,
  '0.140*"bigdata" + 0.045*"ai" + 0.038*"removed" + 0.

## topic modeling north america dataset

In [22]:
#with genim and mallet
number_topics_2 = 11

In [23]:
# Create Dictionary
id2word_2 = corpora.Dictionary(data_ready_2)

# Create Corpus: Term Document Frequency
corpus_2 = [id2word_2.doc2bow(text) for text in data_ready_2]




In [24]:
lda_model_2 = LdaMallet(path_to_mallet_binary, corpus=corpus_2, num_topics=number_topics_2, id2word=id2word_2)

c:\users\gonza\tesisenv\lib\site-packages\gensim\models\wrappers\ldamallet.py:285: ResourceWarning: unclosed file <_io.BufferedReader name='C:\\Users\\gonza\\AppData\\Local\\Temp\\806975_state.mallet.gz'>
  self.word_topics = self.load_word_topics()


In [25]:
#convert lda mallet model to lda gensim model
lda_model_2 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda_model_2)

In [26]:
pprint(lda_model_2.print_topics())

[(0,
  '0.154*"datum" + 0.062*"user" + 0.051*"facebook" + 0.035*"company" + '
  '0.029*"share" + 0.025*"data" + 0.025*"give" + 0.023*"information" + '
  '0.023*"sell" + 0.021*"personal"'),
 (1,
  '0.110*"trump" + 0.022*"campaign" + 0.022*"election" + 0.015*"vote" + '
  '0.013*"work" + 0.012*"retweete" + 0.011*"country" + 0.008*"russian" + '
  '0.008*"win" + 0.008*"child"'),
 (2,
  '0.217*"remove" + 0.193*"link" + 0.032*"today" + 0.029*"watch" + '
  '0.026*"live" + 0.017*"check" + 0.016*"video" + 0.015*"week" + 0.014*"story" '
  '+ 0.011*"testify"'),
 (3,
  '0.435*"remove" + 0.401*"link" + 0.007*"socialmedia" + 0.006*"mining" + '
  '0.004*"network" + 0.003*"rise" + 0.003*"trading" + 0.003*"alt" + '
  '0.003*"signal" + 0.003*"decentralize"'),
 (4,
  '0.038*"good" + 0.036*"question" + 0.031*"thing" + 0.022*"time" + '
  '0.020*"hear" + 0.019*"work" + 0.018*"bad" + 0.017*"zuckerberg" + '
  '0.013*"make" + 0.013*"hearing"'),
 (5,
  '0.401*"remove" + 0.376*"link" + 0.023*"read" + 0.012*"job" 

## Get most relevant documents - Europe dataset

In [27]:
#should I use text or text_completo column? in the first, username and link are not removed

In [28]:
'''
See the discusion here:
https://stackoverflow.com/questions/23509699/understanding-lda-transformed-corpus-in-gensim/37708396?noredirect=1#comment77429460_37708396
https://stackoverflow.com/questions/45310925/how-to-get-a-complete-topic-distribution-for-a-document-using-gensim-lda
'''

'\nSee the discusion here:\nhttps://stackoverflow.com/questions/23509699/understanding-lda-transformed-corpus-in-gensim/37708396?noredirect=1#comment77429460_37708396\nhttps://stackoverflow.com/questions/45310925/how-to-get-a-complete-topic-distribution-for-a-document-using-gensim-lda\n'

In [29]:
#with this code we get the full matrix of topic-documents contribution
matrix_documents_topic_contribution_1, _ = lda_model_1.inference(corpus_1)
matrix_documents_topic_contribution_1 /= matrix_documents_topic_contribution_1.sum(axis=1)[:, None]

In [30]:
matrix_documents_topic_contribution_1 = pd.DataFrame(matrix_documents_topic_contribution_1)
matrix_documents_topic_contribution_1.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.103457,0.068377,0.153100,0.080369,0.087647,0.080369,0.099345,0.068182,0.087511,0.091275,0.080369
1,0.080736,0.076415,0.073314,0.073328,0.100784,0.073314,0.170088,0.076284,0.080745,0.105558,0.089435
2,0.095084,0.089647,0.104423,0.085971,0.097237,0.085763,0.085763,0.089472,0.095112,0.085763,0.085763
3,0.088574,0.077576,0.107303,0.107298,0.083477,0.084574,0.090909,0.094552,0.100313,0.074516,0.090909
4,0.093338,0.087986,0.084175,0.084175,0.095428,0.084175,0.121212,0.087815,0.093346,0.084175,0.084175


In [31]:
contents_1 = pd.Series(df_1['text']).reset_index(drop=True)

In [32]:
matrix_documents_topic_contribution_1 = pd.concat([matrix_documents_topic_contribution_1, contents_1], axis=1)


In [33]:
matrix_documents_topic_contribution_1.head()

,0,1,2,3,4,5,6,7,8,9,10,text
0,0.103457,0.068377,0.153100,0.080369,0.087647,0.080369,0.099345,0.068182,0.087511,0.091275,0.080369,Meet our newest product: the transparent CamHa...
1,0.080736,0.076415,0.073314,0.073328,0.100784,0.073314,0.170088,0.076284,0.080745,0.105558,0.089435,@SupportOurLefty This week has been the most #...
2,0.095084,0.089647,0.104423,0.085971,0.097237,0.085763,0.085763,0.089472,0.095112,0.085763,0.085763,Pulitzer prize for this lady. Investigative jo...
3,0.088574,0.077576,0.107303,0.107298,0.083477,0.084574,0.090909,0.094552,0.100313,0.074516,0.090909,"Theres no more secrecy, confident and privacy ..."
4,0.093338,0.087986,0.084175,0.084175,0.095428,0.084175,0.121212,0.087815,0.093346,0.084175,0.084175,AggregateIQ: the obscure Canadian tech firm an...


## Get most relevant documents - North america dataset

In [34]:
#with this code we get the full matrix of topic-documents contribution
matrix_documents_topic_contribution_2, _ = lda_model_2.inference(corpus_2)
matrix_documents_topic_contribution_2 /= matrix_documents_topic_contribution_2.sum(axis=1)[:, None]

In [35]:
matrix_documents_topic_contribution_2 = pd.DataFrame(matrix_documents_topic_contribution_2)
matrix_documents_topic_contribution_2.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.109091,0.142397,0.081470,0.088499,0.075770,0.087492,0.078918,0.109070,0.075758,0.075758,0.075779
1,0.079748,0.117736,0.085194,0.092003,0.079745,0.110900,0.101053,0.094386,0.079745,0.079745,0.079745
2,0.109091,0.086211,0.081469,0.088458,0.076688,0.087431,0.081405,0.079692,0.075758,0.130202,0.103596
3,0.084175,0.084175,0.084184,0.139712,0.084175,0.084175,0.084175,0.084175,0.102703,0.084175,0.084175
4,0.109091,0.142397,0.081469,0.088497,0.075770,0.087494,0.078917,0.109070,0.075758,0.075758,0.075779


In [36]:
contents_2 = pd.Series(df_2['text']).reset_index(drop=True)

In [37]:
matrix_documents_topic_contribution_2 = pd.concat([matrix_documents_topic_contribution_2, contents_2], axis=1)


In [38]:
matrix_documents_topic_contribution_2.head()

,0,1,2,3,4,5,6,7,8,9,10,text
0,0.109091,0.142397,0.081470,0.088499,0.075770,0.087492,0.078918,0.109070,0.075758,0.075758,0.075779,@SenTedCruz Hold Facebook accountable for abus...
1,0.079748,0.117736,0.085194,0.092003,0.079745,0.110900,0.101053,0.094386,0.079745,0.079745,0.079745,Both sides should just drop it. Trump won...el...
2,0.109091,0.086211,0.081469,0.088458,0.076688,0.087431,0.081405,0.079692,0.075758,0.130202,0.103596,$AAPL lobbing threes while they can. Wasnt it ...
3,0.084175,0.084175,0.084184,0.139712,0.084175,0.084175,0.084175,0.084175,0.102703,0.084175,0.084175,15 min #RSI Signals:\n\n$BTC - $SLR: 6.84\n$BT...
4,0.109091,0.142397,0.081469,0.088497,0.075770,0.087494,0.078917,0.109070,0.075758,0.075758,0.075779,@SenJohnThune Hold Facebook accountable for ab...


# Topic similarity metric

In [39]:
# Choose the # top keywords and # top documents a considerar en la metrica

topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6 

ruta_word_embedding = 'data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [40]:
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
topic_similarity_matrix_multicorpora = vis.calculate_topic_similarity_on_multi_corpora(word_embedding_model, lda_model_1,lda_model_2, corpus_1,corpus_2, id2word_1,id2word_2, matrix_documents_topic_contribution_1,matrix_documents_topic_contribution_2, topn_terms, topk_documents, relevance_lambda)

Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False
Calculating for different omegas


### Show visualization - Multi corpora

In [41]:
#import topicvisexplorer
#import importlib
importlib.reload(topicvisexplorer)

vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
vis.prepare_multi_corpora( lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2,  matrix_documents_topic_contribution_1, matrix_documents_topic_contribution_2, topic_similarity_matrix_multicorpora)

Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False


In [42]:
#save data
vis.save_multi_corpora_data("models_output/multi_corpora_data_europe_northamerica_ca_lda_mallet_gensim.pkl")

Multi corpora data saved sucessfully


In [ ]:
'''
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)

vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
vis.load_multi_corpora_data("multi_corpora_data_europe_northamerica_ca_lda_mallet_gensim.pkl")
'''

In [ ]:
'''
vis.run()
'''

# Topic similarity metric baseline


In [43]:
# Choose the # top keywords and # top documents a considerar en la metrica

topn_terms = 20
topk_documents = 20
relevance_lambda = 0.6 

ruta_word_embedding = 'data/embedding_english_europe_northamerica_word2vec_300dimensions_cbow_trim3_epoch50.model'
word_embedding_model = gensim.models.Word2Vec.load(ruta_word_embedding)

In [44]:
import topicvisexplorer
import importlib
importlib.reload(topicvisexplorer)
warnings.filterwarnings('ignore')
vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
topic_similarity_matrix_multicorpora_metric_baseline = vis.calculate_topic_similarity_on_multi_corpora_metric_baseline(word_embedding_model,  lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2, relevance_lambda, topn_terms)



Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False


In [45]:
#import topicvisexplorer
#import importlib
importlib.reload(topicvisexplorer)

vis = topicvisexplorer.TopicVisExplorer("borrar_nombre")
vis.prepare_multi_corpora( lda_model_1,lda_model_2, corpus_1, corpus_2, id2word_1,id2word_2,  matrix_documents_topic_contribution_1, matrix_documents_topic_contribution_2, topic_similarity_matrix_multicorpora_metric_baseline)

Using Prepared Data script - sort_topics False
Using Prepared Data script - sort_topics False


In [46]:
#save data
vis.save_multi_corpora_data("models_output/multi_corpora_data_europe_northamerica_ca_lda_mallet_gensim_topic_similarity_baseline.pkl")

Multi corpora data saved sucessfully
